In [ ]:
!pip install folium

In [ ]:
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
data_info = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V")
soup = BeautifulSoup(data_info.text, 'html5lib')

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if "Vancouver" not in row.span.text:
        pass
    else:
        cell['PostalCode'] = row.b.text[:3]
        cell['Borough'] = (row.span.a.text).split('(')[0]
        my_string = row.span.a.text
        cell['Neighborhood'] = (((((row.span.text).split(my_string)[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip('( ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

In [ ]:
df.shape

In [ ]:


df['Latitude'] = 0.0
df['Longitude'] = 0.0



In [ ]:
for index, row in df.iterrows():
    address = (str([i for i in row['Neighborhood'].split(',')][0]), str(row['Borough']))
    
    geolocator = Nominatim(user_agent="vancouver_explorer")
    location = geolocator.geocode(address)
    if(location is not None):
        latitude = location.latitude
        df['Latitude'][index] = latitude
        longitude = location.longitude
        df['Longitude'][index] = longitude

In [ ]:
df = df[df["Latitude"] != 0.0]
df.drop_duplicates(subset ="Latitude", keep = False, inplace = True)
df = df.reset_index(drop=True)
df

In [ ]:
df.shape

In [ ]:
geolocator = Nominatim(user_agent="vancouver_explorer")
location = geolocator.geocode("Vancouver, Canada")
latitude = location.latitude
longitude = location.longitude
vancouver_map = folium.Map(location=[latitude, longitude], zoom_start=12)
vancouver_map

In [ ]:
# create map of New York using latitude and longitude values
vancouver_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(vancouver_map)  
    
vancouver_map

In [ ]:
CLIENT_ID = 'WXDDCYBSUHBDMYUR0RVZGAP3N450CE443MAOXJXUFCSI40LI' # your Foursquare ID
CLIENT_SECRET = 'VL302DLWYYPSVYQROKFONGDNO4IE1ZMXZ25FT4CTS4S51R1M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
vancouver_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [ ]:
vancouver_venues.shape

In [ ]:
vancouver_venues.head(10)

In [ ]:
vancouver_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot['Neighborhood'] = vancouver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

In [ ]:
restaurant_data = vancouver_venues[vancouver_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
restaurant_data

In [ ]:
list(restaurant_data["Venue Category"].unique())

In [ ]:
restaurant_data.shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10)) 
restaurants_neigh = (restaurant_data["Neighborhood"].value_counts().sort_index()).plot.barh()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10)) 
restaurants_neigh = (restaurant_data["Venue Category"].value_counts().sort_index()).plot.barh()

In [ ]:
data_without_fastfood = restaurant_data[restaurant_data['Venue Category'] != 'Fast Food Restaurant']
international_restaurants = data_without_fastfood[data_without_fastfood['Venue Category'] != 'Restaurant']

In [ ]:
international_restaurants["Neighborhood"].value_counts()

In [ ]:
plt.figure(figsize=(10,10)) 
restaurants_international = (international_restaurants["Venue Category"].value_counts().sort_index()).plot.barh()

In [ ]:
international_restaurants = international_restaurants[international_restaurants['Venue Category'] != 'Mediterranean Restaurant']
international_restaurants = international_restaurants[international_restaurants['Venue Category'] != 'Italian Restaurant']

international_restaurants = international_restaurants.reset_index(drop=True)
international_restaurants

In [ ]:
international_restaurants.shape

In [ ]:
# create map of New York using latitude and longitude values
vancouver_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(international_restaurants['Venue Latitude'], international_restaurants['Venue Longitude'], international_restaurants['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(vancouver_map)  
    
vancouver_map

In [ ]:
plt.figure(figsize=(10,10)) 
international_neigh = (international_restaurants["Neighborhood"].value_counts().sort_index()).plot.barh()

In [ ]:
plt.figure(figsize=(10,10)) 
restaurants_international = (international_restaurants["Venue Category"].value_counts().sort_index()).plot.barh()

In [ ]:
international_restaurants["Neighborhood"].value_counts().sort_index()

In [ ]:
import matplotlib.pyplot as plt
intervals = [0, 10, 20, 30,40]
plt.figure(figsize=(10,10)) 

plt.hist(x=international_restaurants["Neighborhood"].value_counts(), bins=intervals, rwidth=0.85,)
plt.xlabel('Number of restaurants')
plt.ylabel('Number of Neighborhoods')
plt.xticks(intervals)

plt.show()



In [ ]:
numberOfRestFilter = (international_restaurants['Neighborhood'].value_counts() <= 20) & (international_restaurants['Neighborhood'].value_counts() >= 10)
potential_locations = international_restaurants.groupby('Neighborhood').count()
potential_locations['In interval'] = numberOfRestFilter
potential_locations = potential_locations[potential_locations['In interval'] != False]
potential_locations

In [ ]:
international_restaurants = international_restaurants[(international_restaurants['Neighborhood'] == 'East Fairview, South Cambie') | (international_restaurants['Neighborhood'] == 'North Grandview-Woodland') | (international_restaurants['Neighborhood'] == 'SE West End, Davie Village') | (international_restaurants['Neighborhood'] == 'Waterfront, Coal Harbour, Canada Place')]
international_restaurants.reset_index(drop=True)

In [ ]:
# one hot encoding
vancouver_onehot2 = pd.get_dummies(international_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot2['Neighborhood'] = international_restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vancouver_onehot2.columns[-1]] + list(vancouver_onehot2.columns[:-1])
vancouver_onehot2 = vancouver_onehot2[fixed_columns]

In [ ]:
vancouver_onehot2.shape

In [ ]:
vancouver_onehot2 = vancouver_onehot2.groupby('Neighborhood').sum()
vancouver_onehot2

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = vancouver_onehot2.index


for ind in np.arange(vancouver_onehot2.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_onehot2.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
 create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to the map
markers_colors = []
for index, lat, lon, name, poi in zip(
        international_restaurants.index,
        international_restaurants['Venue Latitude'], 
        international_restaurants['Venue Longitude'], 
        international_restaurants['Venue'], 
        international_restaurants['Neighborhood']):
    label = folium.Popup(str(index), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=name,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.8).add_to(map_clusters)

markers_colors = []
for index, lat, lon, poi in zip(
        international_restaurants.index,
        international_restaurants['Neighborhood Latitude'], 
        international_restaurants['Neighborhood Longitude'], 
        international_restaurants['Neighborhood']):
    label = folium.Popup(str(index), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=100,
        popup=poi,
        color="blue",
        fill=True,
        fill_opacity=0.02).add_to(map_clusters)
    
map_clusters